# 0. 로그인한 유저의 장르 매트릭스를 만듦( 얘가 뭘 주로 보는 앤가 파악)

# 1. 네이버 뉴스검색 api사용해서 키워드 검색
# 2. 긁어온 뉴스에 대해 만든 엔진써서 장르분류
# 3. 장르 매트릭스에 따라 자주 보는 장르를 위로 올림
(predict or predict_proba, 확률기준이 더 복잡함)

1. 분류됬을 때 확률 고만고만한 애들은 뒤로 보냄
2. user_pref에 따라 추천할 때 확률이 높은 애들부터 추천

path = [('정치',0), ('경제',1), ('사회',2), ('세계',3), ('생활문화',4), ('IT과학',5)]

In [1]:
import MySQLdb as db
import pandas as pd
import numpy as np
from naver_news_search import api_search

In [2]:
db = db.connect(
    "127.0.0.1",
    "root",
    '5555',
    "news_rec",
    charset='utf8',
)

In [3]:
view = pd.read_sql('SELECT * FROM VIEW', db)
view['count'] = 1
article = pd.read_sql('SELECT * FROM article', db)

merged = pd.merge(view, article,left_on='article_id', right_on='article_id')
merged = merged.loc[:,['user_id','article_id','area','count']]

genre_matrix = merged.pivot_table(values='count',index='user_id',columns='area',aggfunc=np.sum)

In [218]:
genre_matrix.head()

area,0,1,2,3,4,5
user_id,,,,,,
1,13,11,6,12,11,11
2,9,10,12,6,7,11
3,7,7,13,9,14,14
4,17,10,13,10,12,15
5,11,19,12,11,14,11


In [167]:
data = api_search('파이썬')['items']

In [168]:
crawling_list = []
for each_news in data:
    crawling_list.append(each_news['link'])

In [169]:
def go_get_article(crawling_list):
    import pandas as pd
    import time
    from bs4 import BeautifulSoup
    import requests

    
    
    article_list = []

    for idx, val in enumerate(crawling_list):
        html = val
        response = requests.get(html)
        web_elem = BeautifulSoup(response.content, 'html.parser')
        p_tags = web_elem.find_all('p')

        article = ''

        for each in p_tags:
            article += each.text

        article_list.append(article)

        if idx%10==0:
            print('{} completed, {} total'.format(idx+1,len(crawling_list)))
        
    return article_list

In [377]:
tmp = go_get_article(crawling_list)

1 completed, 100 total
11 completed, 100 total
21 completed, 100 total
31 completed, 100 total
41 completed, 100 total
51 completed, 100 total
61 completed, 100 total
71 completed, 100 total
81 completed, 100 total
91 completed, 100 total


In [383]:
tmp = [each.split('.') for each in tmp]

In [384]:
import re

def trimming(articles):
    is_email = []
    for each in articles:
        tmp = re.findall('[^@]+@[^@]+\.[^@]+', each)
        is_email.append(tmp)
    
    is_email = [True if len(each)!=0 else False for each in is_email]
    upper_bound = len(is_email) - 2 - is_email[::-1].index(True) if True in is_email else len(is_email)-1
    
    articles = [articles[idx] if not is_email[idx] else '' for idx in range(0,upper_bound)]
    
    # 2.
    for idx, val in enumerate(articles):
        converted = re.sub('[^가-힣0-9a-zA-Z.\\s]', ' ', val)
        articles[idx] = converted
    
    # 3.
    articles = [each for each in articles if each != '']
    
    return articles

In [385]:
tmp = [''.join(trimming(each)) for each in tmp]

In [386]:
import pickle
model = pickle.load(open('dataset/twitter-tfidf-mulnb.pkl','rb'))

# 확률이 전부 고만고만한 애들은 제외

In [387]:
news_content = pd.DataFrame(columns=['content'])
news_content['content'] = pd.Series(tmp)

In [388]:
news_content.head()

,content
0,청년 실업문제가 장기화되며 수요가 많은 유망직종에 대한 취준생의 관심이 나날이 확대...
1,\n\n\n 메가스터디교육그룹 빅데이터 전문가 양성과정 수강생 중 인문사회계열 출...
2,신유경인턴 기자 최근 이러한 데이터의 중요성에 대해서 각 기업들도 분주하다 ...
3,\n해당 언론사가 채널 주요 뉴스로직접 선정한 기사입니다\n채널 안내\n닫기\n설정...
4,\r\n\t\t\t 아시아뉴스통신 김정수기자 \r\n\t\t\t기사입력 201...


In [389]:
X_test = news_content['content']

In [391]:
%%time
y_pred = model.predict(X_test)

Wall time: 2.99 s


In [392]:
# 키워드가 파이썬인데 사회로 분류된 애들
y_pred.argsort()[:5]

array([65, 76, 68, 41, 42], dtype=int64)

In [393]:
y_pred[76]

1

In [394]:
y_pred[68]

1

In [395]:
genre_matrix.loc[1]

area
0    13
1    11
2     6
3    12
4    11
5    11
Name: 1, dtype: int64

In [374]:
def news_rank(user_id, genre_matrix, y_pred):
    user_pref = genre_matrix.loc[user_id].sort_values(ascending=False).index
    
    print(user_pref,'\n')
    
    article = []
    
    for area in range(0,6):
        each_area = np.argwhere(y_pred==area)
        each_area = each_area.reshape(1,len(each_area))
        article.append(each_area)
    
    news_rank = []

    for idx, each in enumerate(article):
        print(idx, each)
    
    for pref_area in user_pref:
        for each_article in article[pref_area]:
            for each in each_article:
                news_rank.append(each)
    
    return news_rank

In [375]:
news = news_rank(1,genre_matrix, y_pred)

Int64Index([0, 3, 5, 4, 1, 2], dtype='int64', name='area') 

0 [[65]]
1 [[41 42 68 76]]
2 [[ 4 49 50 52]]
3 []
4 [[ 3  9 12 17 22 24 26 28 29 30 31 34 36 43 45 47 51 53 63 66 71 73 82 84
  85 89 90 92 94 95 96]]
5 [[ 0  1  2  5  6  7  8 10 11 13 14 15 16 18 19 20 21 23 25 27 32 33 35 37
  38 39 40 44 46 48 54 55 56 57 58 59 60 61 62 64 67 69 70 72 74 75 77 78
  79 80 81 83 86 87 88 91 93 97 98 99]]


In [397]:
for each in news:
    print(crawling_list[each])

http://it.chosun.com/news/article.html?no=2849103
http://www.getnews.co.kr/news/articleView.html?idxno=62918
http://www.metroseoul.co.kr/news/newsview?newscd=2018041500072
http://edu.donga.com/?p=article&ps=view&at_no=20180413175338210274
http://www.newsbrite.net/news/view.php?no=14914
http://www.dailygrid.net/news/articleView.html?idxno=73756
http://www.itworld.co.kr/news/108910
http://www.enewstoday.co.kr/news/articleView.html?idxno=1178865
http://www.hellot.net/new_hellot/magazine/magazine_read.html?code=202&sub=004&idx=40459
http://www.cctvnews.co.kr/news/articleView.html?idxno=82039
http://edu.donga.com/?p=article&ps=view&at_no=20180412154229620376
http://www.ciobiz.co.kr/news/article.html?id=20180412120006
http://www.itnews.or.kr/?p=26296
http://www.newsworks.co.kr/news/articleView.html?idxno=179657
http://www.ciokorea.com/news/37897
http://www.betanews.net:8080/article/838441.html
http://www.itdaily.kr/news/articleView.html?idxno=88551
http://www.greened.kr/news/articleView.html